In [58]:
import numpy as np
import pandas as pd


from sklearn import preprocessing
from datetime import datetime

import pickle
import joblib


import os
import sys 

import pyodbc

from datetime  import datetime

# Log Error  

In [69]:
def WriteLogError(error):
        f = open('ml_error.csv', 'a')
        error_str=f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}|{repr(error)}\n'
        f.write(error_str)
        f.close()
        print(error_str) 
        raise error

# Check the most critical variables

In [70]:
#x_date=datetime(2020,5,11,9,45,0)
x_date=datetime.now()


isRunByscheduling=False

onlyLastRecord=False

isSavedDB=False
#CONSTANT VARIABLE
predictionColName="Signal"

strategyName='long'


if isRunByscheduling :
 n = len(sys.argv) 
 print("Total arguments passed:", n) 

 try:

  if n==2:
    strategyName= sys.argv[1]
  else  :
    raise Exception("you must specify the strategyName parameter either of long or short") 
 except Exception as error:
    WriteLogError(error) 

print(f"ML-StrategyTrade={strategyName}")

ML-StrategyTrade=long


# Set all Path Files

In [79]:
holiday_path=os.path.abspath(r'E:\PredictML-Script\S50F15M_FeaturesExploration\ChkHolidayForYear.csv')

if onlyLastRecord:
  #Live Run with last data
 abfeatures_file='S50M15_ListFeatures.csv'
 ml_file_path='E:\\PredictML-Script\\S50F15M_FeaturesExploration\\UnlabeledData\\'

else:
#Test Run with bulk data
 abfeatures_file='S50M15_ListFeatures_MultipleRows.csv'
 ml_file_path='E:\\PredictML-Script\\S50F15M_FeaturesExploration\\UnlabeledData\\'



# Set Model File Path

In [80]:
if strategyName=='long':
 modelfile=os.path.abspath(r'E:\PredictML-Script\Models\long_S50M15_XGB-160x8x01_01-2007t12-2018_b100520_1139.joblib.dat')
 trade_symbol='LONG_S50IF'
else:
 modelfile=os.path.abspath(r'E:\PredictML-Script\Models\short_S50M15_XGB-160x8x01_01-2007t12-2018_b100520_1138.joblib.dat')
 trade_symbol='SHORT_S50IF'
 
print(trade_symbol)

LONG_S50IF


# Function For Check Market Time

In [81]:
def CheckValidPath(path):
        if os.path.exists(path):
            return True
        else:
            raise Exception(f'Invalid path: {path}') 

In [82]:
# 10:01 is before the first triger in the morning  at 10:02  for first bar at 10:00
# 12:26 is after the last trigter in the morning at 12:17 for first bar at 12:15

# 14:31 is before the first triger after noon  at 14:32  for first bar at 14:30
# 16:56 is after the last trigter after noon at 16:47 for first bar at 16:45

def CheckMarketTime(current_time):
  # check time 10:00-17:00
  isTradeTime_morning=current_time.strftime('%H:%M:%S')>=10:01:00' and current_time.strftime('%H:%M:%S')<='12:36:00' 
  isTradeTime_noon=current_time.strftime('%H:%M:%S')>='14:31:00' and current_time.strftime('%H:%M:%S')<='17:06:00' 
  
  return  isTradeTime_morning or isTradeTime_noon
    

In [83]:
def CheckMarketDay(current_time):
   w_no= int(current_time.strftime('%w'))
   if w_no>=1 and w_no<=5 :
      return True
   else :
      return False

In [84]:
def CheckNotHoliday(df_holiday,current_time):

 holiday_today=current_time.strftime('%Y%m%d')

 df_holiday[0]= current_time.strftime('%Y')+df_holiday[0]
 #print(df_holiday)

 if holiday_today not in df_holiday[0].tolist():
  return True
 else:
  return False
    

# Check Trading Time&Date

In [85]:

print("Current time")
print(x_date.strftime('%d/%m/%Y'))
print(x_date.strftime('%H:%M:%S'))
#print(x_today)

print("===========check trade time&day===========")

IsMarketTime=CheckMarketTime(x_date)
print('IsMarkerTime = ',CheckMarketTime(x_date))

IsMarketDay=CheckMarketDay(x_date)
print('IsMarketDay = ',CheckMarketDay(x_date))


try:
 df_holiday =pd.read_csv(holiday_path,header =  None,dtype=object)

 IsNotHoliday=CheckNotHoliday(df_holiday ,x_date)
 print('IsNotholiday = ',IsNotHoliday)


 IsTradable=IsMarketTime and  IsMarketDay and  IsNotHoliday
 if not IsTradable:
   sys.exit(("It is not trading time")) 


 print("#######################################################")
 print("Let go it is time to trade")

except Exception as error:
 WriteLogError(error) 
    

Current time
11/05/2020
09:45:00
===========check trade time&day===========
IsMarkerTime =  True
IsMarketDay =  True
IsNotholiday =  True
#######################################################
Let go it is time to trade


# Check all Path Files

In [88]:
predicttime=datetime.now().strftime('-p%d%m%y_%H%M')
prediction_file=f'{strategyName}{predictionColName}_{predicttime}_{abfeatures_file}'


path_abfeatures= os.path.abspath(f'{ml_file_path}{abfeatures_file}')
path_prediction=ml_file_path+prediction_file


try:
    if CheckValidPath(modelfile) and CheckValidPath(ml_file_path) and CheckValidPath(path_abfeatures) and CheckValidPath(holiday_path)  :
       print("Model File= ",modelfile)
       print("Feature Path= ",ml_file_path)
       print("New Data Feature File= ",path_abfeatures)
       print("Holiday Path= ",holiday_path)
     
except Exception as error:
       WriteLogError(error)                                                                                
                                                                                    

Model File=  E:\PredictML-Script\Models\long_S50M15_XGB-160x8x01_01-2007t12-2018_b100520_1139.joblib.dat
Feature Path=  E:\PredictML-Script\S50F15M_FeaturesExploration\UnlabeledData\
New Data Feature File=  E:\PredictML-Script\S50F15M_FeaturesExploration\UnlabeledData\S50M15_ListFeatures_MultipleRows.csv
Holiday Path=  E:\PredictML-Script\S50F15M_FeaturesExploration\ChkHolidayForYear.csv


# 1.Load unseen data file to predict by model

In [89]:

try:
    dataset =pd.read_csv(path_abfeatures)
    
    #For one record
    #if onlyLastRecord==True:
     #dataset =dataset.tail(1)
    
    if dataset.shape[0]>0:
        print(f'1-Load unseen data file from {path_abfeatures}  succeeded.')
        dataset['Ticker']=trade_symbol
    else:
        raise Exception(f'No feature row  in dataframe from file,please check Amibroker Exploration')    
    
except Exception as error_x:
        WriteLogError(f'{error_x} {path_abfeatures}')


print(dataset.info())


1-Load unseen data file from E:\PredictML-Script\S50F15M_FeaturesExploration\UnlabeledData\S50M15_ListFeatures_MultipleRows.csv  succeeded.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ticker                  22 non-null     object 
 1   Date/Time               22 non-null     object 
 2   open                    22 non-null     float64
 3   high                    22 non-null     float64
 4   low                     22 non-null     float64
 5   close                   22 non-null     float64
 6   indy_ma-550             22 non-null     float64
 7   indy_ma-1100            22 non-null     float64
 8   indy_hh-550             22 non-null     float64
 9   indy_ll-550             22 non-null     float64
 10  indy_mid-550            22 non-null     float64
 11  indy_hh2-1100           22 non-null     float64
 12  indy_ll2-1

# 2.Create dataframe succeeded

In [90]:
noFeature_col=['Ticker','Date/Time' ,'open','high','low','close']
dfx_ohlc=dataset[noFeature_col]
dfx_feature=dataset.drop(columns=noFeature_col)
print('2-Create dataframe succeeded.')

2-Create dataframe succeeded.


In [91]:
print(dfx_ohlc.tail(3))

        Ticker           Date/Time   open   high    low  close
19  LONG_S50IF  11/5/2020 16:15:00  862.3  862.3  860.6  861.5
20  LONG_S50IF  11/5/2020 16:30:00  861.4  861.7  860.1  860.4
21  LONG_S50IF  11/5/2020 16:45:00  860.5  860.5  858.2  860.3


In [92]:
print(dfx_feature.tail(3))

    indy_ma-550  indy_ma-1100  indy_hh-550  indy_ll-550  indy_mid-550  \
19       826.95        806.26        873.4        728.0         800.7   
20       827.18        806.20        873.4        728.0         800.7   
21       827.39        806.14        873.4        728.0         800.7   

    indy_hh2-1100  indy_ll2-1100  indy_mid2-1100  indy_macd110-440  \
19          946.2          623.0           784.6             14.34   
20          946.2          623.0           784.6             14.30   
21          946.2          623.0           784.6             14.27   

    indy_signal110-440-110  indy_hist_macd110-440  indy_rsi25-ma20  \
19                   20.44                  -6.10            63.80   
20                   20.32                  -6.02            63.69   
21                   20.20                  -5.93            63.60   

    indy_6ATRTrail_DC-110  cate_3trend-550_ma110  cate_2trend-1100_ma220  \
19                 852.99                      3                     

# 3.Create numpy array as input for model

In [93]:
X_new=dfx_feature.values
print(X_new[:3,:])
print('3-Create numpy array succeeded.')


[[822.87 807.26 873.4  728.   800.7  946.2  623.   784.6   15.41  22.88
   -7.47  46.3  852.24   3.     0.     4.     3.  ]
 [823.07 807.2  873.4  728.   800.7  946.2  623.   784.6   15.28  22.75
   -7.47  47.06 852.24   3.     0.     4.     3.  ]
 [823.28 807.14 873.4  728.   800.7  946.2  623.   784.6   15.16  22.62
   -7.46  47.92 852.16   3.     0.     4.     3.  ]]
3-Create numpy array succeeded.


# 4.Load model file as strategy for prediction

In [94]:
 try:  
   loaded_model = joblib.load(modelfile)
  
   #loaded_model =  pickle.load(open(modelfile, "rb"))
  
   print(f"4-Loaded model from: {modelfile} succeeded")
 except Exception as error:
   WriteLogError(f'cannot load file {modelfile} {error}')  

4-Loaded model from: E:\PredictML-Script\Models\long_S50M15_XGB-160x8x01_01-2007t12-2018_b100520_1139.joblib.dat succeeded


# 5.Predict unseen feature data

In [95]:
yPredited = loaded_model.predict(X_new)
predictions = [round(value) for value in yPredited]

prediction_df=pd.DataFrame(data= {predictionColName: predictions })

df_result=pd.concat([dfx_ohlc,prediction_df],axis=1)
print('5-Predict new data succeeded.')

5-Predict new data succeeded.


In [96]:
print("Sample Predict result")
print("=======================================================================")
print(df_result[:5])

Sample Predict result
       Ticker           Date/Time   open   high    low  close  Signal
0  LONG_S50IF   11/5/2020 9:45:00  852.0  854.5  851.5  854.2       0
1  LONG_S50IF  11/5/2020 10:00:00  854.3  856.3  852.8  853.9       0
2  LONG_S50IF  11/5/2020 10:15:00  854.0  858.8  853.8  858.0       0
3  LONG_S50IF  11/5/2020 10:30:00  858.1  861.9  857.8  861.6       0
4  LONG_S50IF  11/5/2020 10:45:00  861.5  862.2  859.0  860.8       0


# 6.1 Save To CSV

In [97]:
if isSavedDB==False:
 df_result.drop(columns=['Ticker'],inplace=True)
 df_result.to_csv(path_prediction,index=False)
 print('6-Save predction result to csv succeeded.')


# 6.2 Save To DB (check database server and database name)

In [1]:
try:
 if isSavedDB==True:
  conn = pyodbc.connect('Driver={SQL Server};'
                       'Server=localhost\SQLEXPRESS;'
                       'Database=Tfex_AB_ML;'
                       'Trusted_Connection=yes;')

  cursor=conn.cursor()

  for index,row in df_result.iterrows():
      #print(datetime.strptime(row["Date/Time"],'%d/%m/%Y %H:%M:%S'))
      #print(row["Ticker"],row["Date/Time"])
      sqlcmd="INSERT INTO S50F_ML ([SYMBOL],[xDATE],[xOPEN],[xHIGH],[xLOW],[xCLOSE],[xVolume],[Signal],[UpdateTime]) VALUES(?,?,?,?,?,?,?,?,?)"
      cursor.execute(sqlcmd,row["Ticker"],datetime.strptime(row["Date/Time"],'%d/%m/%Y %H:%M:%S'),row["open"],row["high"],row["low"],row["close"],0,row[predictionColName],datetime.now())
      conn.commit()
    
  cursor.commit()
  cursor.close()
  conn.close()
  print('6-Save predction result to database succeeded.')
 #=================================================================
except Exception as error :
  WriteLogError(error)  

NameError: name 'WriteLogError' is not defined